In [1]:
# necessary packages #

#using Pkg
#Pkg.add("Distances")
using Distributions
using Random
using Distances
using LinearAlgebra
using SparseArrays
using IterativeSolvers
using ProgressMeter
using JLD2
#using NearestNeighbors # Can't use KDtree in julia for posterior prediction

In [2]:
Sys.free_memory()/(2^20*1024)

19.668025970458984

In [3]:
#using Pkg
#Pkg.add("RData")

In [4]:
include("../../../util2.j")

colnorm (generic function with 1 method)

In [5]:
# unnecessary packages #

#using Pkg
#Pkg.add("UnicodePlots")
using UnicodePlots   # check the structure of the sparse matrix
using BenchmarkTools

using StatsPlots
using MCMCChains
using PrettyTables
using Dates
using StatsBase

In [6]:
#using Pkg
#Pkg.add("StatsBase");
#using NearestNeighborDescent

In [7]:
using RData
data_cleaned = load("../../data/rawdata/data_cleaned_small_expanded.RData", convert = true)["data_cleaned_small"];

In [8]:
countmap(data_cleaned.LC_Type4)

Dict{Float64,Int64} with 7 entries:
  7.0 => 83009
  4.0 => 30094
  2.0 => 8421
  5.0 => 27043
  8.0 => 3111
  6.0 => 296479
  1.0 => 571843

In [9]:
hold_index_set = load("../../data/rawdata/hold_index_small_expanded.RData", convert = true)

Dict{String,Any} with 11 entries:
  "hold_ind4"       => Int32[796395, 815927, 678460, 506073, 78700, 980165, 206…
  "hold_ind10"      => Int32[665387, 373156, 724373, 1019331, 888091, 918487, 8…
  "hold_ind2"       => Int32[518549, 906271, 597282, 651904, 872494, 353593, 32…
  "hold_ind6"       => Int32[896510, 543595, 201130, 86952, 630506, 206437, 452…
  "hold_ind5"       => Int32[811779, 351188, 843347, 549899, 712081, 81044, 972…
  "hold_ind9"       => Int32[498700, 871568, 343872, 751268, 699631, 503105, 87…
  "hold_ind7"       => Int32[914319, 272494, 251816, 456856, 273822, 586315, 96…
  "hold_ind3"       => Int32[466194, 761309, 456463, 45604, 728959, 693334, 733…
  "hold_ind1"       => Int32[548676, 452737, 124413, 436523, 856018, 666931, 25…
  "hold_ind8"       => Int32[363116, 896446, 832503, 10248, 932012, 542066, 851…
  "hold_area_indx2" => Int32[431753, 431754, 431755, 431756, 431757, 431758, 43…

In [10]:
U_indx = sort(intersect(hold_index_set["hold_ind1"], hold_index_set["hold_ind2"]));
N = size(data_cleaned)[1];
S_indx = setdiff(1:N, U_indx);

In [11]:
# coordinate:
coords = [data_cleaned.scaled_x[S_indx] data_cleaned.scaled_y[S_indx]];
coords_pred = [data_cleaned.scaled_x[U_indx] data_cleaned.scaled_y[U_indx]];
n = size(coords)[1]; m = 10;

In [12]:
#@load "../data/LMC_data/LMC_data.jld"

In [13]:
Dates.format(now(), "HH:MM:SS")  

"00:17:38"

In [14]:
# build neighbors #
m = 10;                                                       # number of nearest neighbor                       
NN = BuildNN(coords, m, 1.0, 0.0);                       # build nearest neighbor 
nnIndx_col = vcat(NN.nnIndx, 1:n);                            # the index of columns
nnIndx_row = zeros(Int64, 0);                                               
for i in 2:m
    nnIndx_row = vcat(nnIndx_row, fill(i, i-1))
end
nnIndx_row = vcat(nnIndx_row, repeat((m + 1):n, inner = m), 1:n);  # the index of rows

In [15]:
Dates.format(now(), "HH:MM:SS")  

"00:51:55"

In [16]:
# construct Atilde Dtilde #

using RCall
@rput coords
@rput coords_pred
@rput m
R"""
library("RANN")
nn_mod_ho <- nn2(coords, coords_pred, k = m)
"""
@rget nn_mod_ho
NM = size(coords_pred)[1];
MnnIndxLU = collect(1:m:(NM * m + 1));
MnnIndx = vec(nn_mod_ho[:nn_idx]');
Mnndists = vec(nn_mod_ho[:nn_dists]');

┌ Warning: Package RCall does not have AxisArrays in its dependencies:
│ - If you have RCall checked out for development and have
│   added AxisArrays as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with RCall
│ Loading AxisArrays into RCall from project dependency, future warnings for RCall are suppressed.
└ @ nothing nothing:840


In [17]:
Dates.format(now(), "HH:MM:SS")  

"00:52:03"

In [18]:
@save "../../data/LMC_data/LMC_data_small_expanded.jld" U_indx S_indx N n m NN nnIndx_col nnIndx_row NM MnnIndxLU MnnIndx Mnndists coords coords_pred